In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [2]:
# connexion à la première page
url = "https://www.paruvendu.fr/immobilier/annonceimmofo/liste/listeAnnonces?tt=1&at=1&nbp0=99&pa=FR&lo=75,77,78,91,92,93,94,95&lol=0&ray=50"

# Url ancien
url = "https://www.paruvendu.fr/immobilier/annonceimmofo/liste/listeAnnonces?nbp=0&tt=1&tbApp=1&tbDup=1&tbChb=1&tbLof=1&tbAtl=1&tbPla=1&tbMai=1&tbVil=1&tbCha=1&tbPro=1&tbHot=1&tbMou=1&tbFer=1&at=-4&nbp0=99&pa=FR&lo=75,77,78,91,92,93,94,95&ddlFiltres=nofilter"


In [3]:
scrap_list = [[] for x in range(6)]
agence_list = []
def normalize_text(text):
    temp = str(text).replace("\t","")
    temp = str(temp).replace("\n","")
    temp = str(temp).replace("\r"," ")
    return(temp)

In [4]:
def get_data(url, scrap_list): 
    root_url ="https://www.paruvendu.fr"
    r = requests.get(url)
    if r.connection :
        soup = BeautifulSoup(r.text,"lxml")
        
        h3 = soup.find_all("div", {"class":"ergov3-h3"})
        for elem in h3:     
            scrap_list[0].append(normalize_text(elem.span.text))
            scrap_list[1].append(normalize_text(elem.cite.text))
        
        priceannonce = soup.find_all("div", {"class":"ergov3-priceannonce"})
        for elem in priceannonce: 
            scrap_list[2].append(normalize_text(elem.text))
        
        textannonce = soup.find_all("p", {"class":"txt-long"})
        for elem in textannonce: 
            scrap_list[3].append(normalize_text(elem.text))
        
        lienpro= soup.find_all("div", {"class":"ergov3-lienpro_2lines"})
        for elem in lienpro: 
            try :
                scrap_list[4].append(normalize_text(elem.span.text))
            except AttributeError : 
                scrap_list[4].append("PARTICULIER")
                
        link_url = soup.find_all("a", {"class":"voirann"})
        for elem in link_url:
            if ((str(elem["href"]).startswith("#")) or (str(elem["href"]).startswith("j")) or (str(elem["href"]).startswith("'https://www.paruvendu.frhttps://www.paruvendu.fr/n/programme-neuf/"))):
                continue
            else :
                temp_url = root_url+str(elem["href"])
                scrap_list[5].append(temp_url)

    
        
    return(scrap_list)

In [5]:
r = requests.get(url)
soup = BeautifulSoup(r.text,"lxml")
url = "https://www.paruvendu.fr/immobilier/annonceimmofo/liste/listeAnnonces?nbp=0&tt=1&tbApp=1&tbDup=1&tbChb=1&tbLof=1&tbAtl=1&tbPla=1&tbMai=1&tbVil=1&tbCha=1&tbPro=1&tbHot=1&tbMou=1&tbFer=1&at=-4&nbp0=99&pa=FR&lo=75,77,78,91,92,93,94,95&ddlFiltres=nofilter"

options = Options()
options.add_argument("start-maximized")
options.add_argument("--lang=en")
driver = webdriver.Chrome(chrome_options=options, executable_path=r'chromedriver.exe')
driver.get(url)
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH,"/html/body/div[1]/div/div[1]/div[3]/button[2]"))).click()


for i in range(100): 
    sleep(0.5)
    end_url="&p="+str(i+1)
    if i+1 == 1 :
        scrap_list =get_data(url, scrap_list)
    else :
        new_url = url+end_url
        scrap_list =get_data(new_url, scrap_list)

C:\Users\andy\AppData\Local\Temp/ipykernel_1144/1735805891.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_options=options, executable_path=r'chromedriver.exe')
C:\Users\andy\AppData\Local\Temp/ipykernel_1144/1735805891.py:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=r'chromedriver.exe')


In [11]:
df = pd.DataFrame(scrap_list)
df = df.transpose()
df =  df.rename(columns={0: "Biens", 1: "Ville", 2: "Prix" , 3:"Description", 4:"Agence", 5:"Liens URL"})
df

,Biens,Ville,Prix,Description,Agence,Liens URL
0,"Appartement 2 pièces,50 m²",Paris 13,535 000 €,Quartier bnf - 2 pièces vue dégagée. install...,Société NOVAE,https://www.paruvendu.fr/immobilier/vente/appa...
1,"Appartement 3 pièces,65 m²",Ivry-sur-Seine (94200),310 000 €,Charmant duplex. l'agence l'adresse vous prop...,Société NOVAE,https://www.paruvendu.fr/immobilier/vente/appa...
2,"Appartement 3 pièces,54 m²",Nogent-sur-Marne (94130),305 000 €,Appartement nogent sur marne 3 pièce(s). dans...,ADN GESTION TRANSACTION,https://www.paruvendu.fr/immobilier/vente/appa...
3,"Appartement 4 pièces,61 m²",Champigny-sur-Marne (94500),218 000 €,Appartement champigny sur marne 4 pièce(s) 61...,ADJ GESTION,https://www.paruvendu.fr/immobilier/vente/appa...
4,"Appartement 3 pièces,71 m²",Fontenay-sous-Bois (94120),578 000 €,Appartement fontenay sous bois 3 pièce(s) 71 ...,IMMOGEF FONTENAY,https://www.paruvendu.fr/immobilier/vente/appa...
...,...,...,...,...,...,...
2695,"Appartement 2 pièces,40 m²",Parmain (95620),160 000 €,Chambly. a 10 min de parmain appartement f2 o...,None,https://www.paruvendu.fr/immobilier/vente/appa...
2696,"Maison 9 pièces,232 m²",Parmain (95620),440 000 €,Limite lamorlaye maison familiale 230m². a b...,None,https://www.paruvendu.fr/immobilier/vente/mais...
2697,"Appartement 1 pièce,23 m²",Parmain (95620),130 000 €,"Studio auvers sur oise. a auvers sur oise, da...",None,https://www.paruvendu.fr/immobilier/vente/appa...
2698,"Appartement 3 pièces,60 m²",Parmain (95620),199 000 €,Appartement persan. a 15 minutes de parmain s...,None,https://www.paruvendu.fr/immobilier/vente/appa...
